In [38]:
import os
import re
import json
from openai import OpenAI
from dotenv import load_dotenv
from typing import TypedDict, List, Dict, Literal, Any, Callable
from serpapi import SerpApiClient

load_dotenv()

class TextBlock(TypedDict):
    type: Literal["text"]
    text: str

class Message(TypedDict):
    role: Literal["system", "user", "assistant", "tool"]
    content: List[TextBlock]   

In [36]:
class HelloAgenticsLLM:
    def __init__(self, model: str = None, apiKey: str = None, baseUrl: str = None, timeout: int = None):
        self.model = model or os.getenv('LLM_MODEL_ID')
        apiKey = apiKey or os.getenv('LLM_API_KEY')
        baseUrl = baseUrl or os.getenv('LLM_BASE_URL')
        timeout = timeout or int(os.getenv('LLM_TIMEOUT', 60))

        if not all([self.model, apiKey, baseUrl]):
            raise ValueError('模型ID，API密匙和服务地址必须提供')
        
        self.client = OpenAI(api_key=apiKey, base_url=baseUrl, timeout=timeout)

    def safe_stream_collect_text(self, response):
        collected = []

        for chunk in response:
            # --- 情况 1：chunk 是 tuple ---
            if isinstance(chunk, tuple):
                # 常见结构：(event_type, data)
                if len(chunk) < 2:
                    continue
                data = chunk[1]
            else:
                data = chunk

            # --- 情况 2：data 是 dict ---
            if isinstance(data, dict):
                choices = data.get("choices")
                if not choices:
                    continue

                delta = choices[0].get("delta", {})
                text = delta.get("content")

            # --- 情况 3：OpenAI SDK 对象 ---
            else:
                if not hasattr(data, "choices"):
                    continue
                if not data.choices:
                    continue

                choice = data.choices[0]
                if not hasattr(choice, "delta"):
                    continue

                text = getattr(choice.delta, "content", None)

            # --- 最终兜底 ---
            if not isinstance(text, str):
                continue

            # 过滤 think 标签
            if "<think>" in text or "</think>" in text:
                continue

            # print(text, end="", flush=True)
            collected.append(text)

        return "".join(collected)

    def think(self, messages: List[Dict[str, str]], temperature: float = 0) -> str:
        print(f'🧠正字调用 {self.model} 模型...')
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                temperature=temperature,
                stream=True
            )

            print('✅ 大模型响应成功：')
            response_text = self.safe_stream_collect_text(response)
            print(response_text)
            return response_text

                
        except Exception as e:
            print(f"❌ 调用LLM API时发生错误: {e}")
            return None
llmClient = HelloAgenticsLLM()

In [ ]:
input_data = [
    {"role": "system", "content": "You are a helpful assistant that writes Python code."},
    {"role": "user", "content": "写一个快速排序算法"}
]

In [18]:
print("\n\n ---调用LLM---")
responseText = llmClient.think(input_data)
if responseText:
    print("\n\n ---完整模型响应---")
    print(responseText)



 ---调用LLM---
🧠正字调用 GLM-4.1V-Thinking-Flash 模型...
✅ 大模型响应成功：


 ---完整模型响应---
以下是实现**快速排序**算法的 Python 代码，包含详细的注释说明：  


```python
def quick_sort(arr, low=0, high=None):
    """
    快速排序算法
    
    参数:
        arr: 待排序的列表
        low: 子数组的起始索引（默认为0）
        high: 子数组的结束索引（默认为列表末尾）
    
    返回:
        无，直接原地修改列表
    """
    if high is None:
        high = len(arr) - 1  # 如果未指定high，则默认为列表最后一个元素的索引
    
    if low < high:  # 当子数组至少有两个元素时，继续递归
        # 调用 partition 函数获取分区后的基准索引（pivot_index）
        pivot_index = partition(arr, low, high)
        
        # 递归对「左子数组」进行快速排序
        quick_sort(arr, low, pivot_index - 1)
        
        # 递归对「右子数组」进行快速排序
        quick_sort(arr, pivot_index + 1, high)


def partition(arr, low, high):
    """
    分区函数：将子数组划分为「小于基准」和「大于等于基准」的两部分，并返回基准的最终位置
    
    参数:
        arr: 待分区的列表
        low: 子数组的起始索引
        high: 子数组的结束索引
    
    返回:
        基准（pivot）的最终索引
    """
    # 选择「第一个元素」作为基准（也可以选随机元素或中间元素优化性能）
    pivot = arr[low]
    
    # 初始化「较小元素的分界索引」（

## ReAct
通过thought-action-observation方式

Thought (思考)： 这是智能体的“内心独白”。它会分析当前情况、分解任务、制定下一步计划，或者反思上一步的结果。
Action (行动)： 这是智能体决定采取的具体动作，通常是调用一个外部工具，例如 Search['华为最新款手机']。
Observation (观察)： 这是执行Action后从外部工具返回的结果，例如搜索结果的摘要或API的返回值。

## Tool
### Google网页查询工具SerpAPI


In [19]:
def search(query: str) -> str:
    print(f"🔍 正在执行 [SerpApi] 网页搜索: {query}")
    try:
        api_key = os.getenv('SERPAPI_API_KEY')
        if not api_key:
            return "错误:SERPAPI_API_KEY 未在 .env 文件中配置。"
        
        params = {
            'engine': "google",
            'q': query,
            "api_key": api_key,
            'gl': "cn",
            'hl': "zh-cn",
        }

        client = SerpApiClient(params)
        results = client.get_dict()

        # 智能解析:优先寻找最直接的答案
        if "answer_box_list" in results:
            return "\n".join(results["answer_box_list"])
        if "answer_box" in results and "answer" in results["answer_box"]:
            return results["answer_box"]["answer"]
        if "knowledge_graph" in results and "description" in results["knowledge_graph"]:
            return results["knowledge_graph"]["description"]
        if "organic_results" in results and results["organic_results"]:
            # 如果没有直接答案，则返回前三个有机结果的摘要
            snippets = [
                f"[{i+1}] {res.get('title', '')}\n{res.get('snippet', '')}"
                for i, res in enumerate(results["organic_results"][:3])
            ]
            return "\n\n".join(snippets)
        
        return f"对不起，没有找到关于 '{query}' 的信息。"

    except Exception as e:
        return f"搜索时发生错误: {e}"

### 构建通用的工具执行器

In [20]:
class ToolExecutor:
    """
    一个工具执行器，负责管理和执行工具。
    """
    def __init__(self):
        self.tools: Dict[str, Dict[str, Any]] = {}

    def registerTool(self, name:str, description:str, func: Callable):
        """
        向工具箱中注册一个新工具。
        """
        if name in self.tools:
            print(f"警告：工具 {name} 已存在，将被覆盖。")
        self.tools[name] = {"description": description, 'func':func}
        print(f"工具 {name} 已注册。")

    def getTool(self, name: str) -> Callable:
        """
        根据名称获取一个工具的执行函数。
        """

        return self.tools.get(name, {}).get('func')
    
    def getAvaliableTools(self) -> str:
        """
        获取所有可用工具的格式化描述字符串。
        """
        return "\n".join([
            f"- {name}: {info['description']}"
            for name, info in self.tools.items()
        ]) 

In [21]:
toolExecutor = ToolExecutor()

search_description = "一个网页搜索引擎。当你需要回答关于时事、事实以及在你的知识库中找不到的信息时，应使用此工具。"
toolExecutor.registerTool(name="Search", description=search_description, func=search)

print("\n ---可用工具---")
print(toolExecutor.getAvaliableTools())

print("\n ---执行 Action: Search['英伟达最新的GPU型号是什么]---")
tool_name = "Search"
tool_input = "英伟达最新的GPU型号是什么"

tool_function = toolExecutor.getTool(tool_name)
if tool_function:
    observation = tool_function(tool_input)
    print("---观察 （Observation）---")
    print(observation)
else:
    print(f"错误：未找到名为'{tool_name}'的工具")

工具 Search 已注册。

 ---可用工具---
- Search: 一个网页搜索引擎。当你需要回答关于时事、事实以及在你的知识库中找不到的信息时，应使用此工具。

 ---执行 Action: Search['英伟达最新的GPU型号是什么]---
🔍 正在执行 [SerpApi] 网页搜索: 英伟达最新的GPU型号是什么
---观察 （Observation）---
[1] 比较GeForce 系列最新一代显卡和前代显卡
比较最新一代RTX 30 系列显卡和前代的RTX 20 系列、GTX 10 和900 系列显卡。查看规格、功能、技术支持等内容。

[2] GeForce 显卡| NVIDIA
探索NVIDIA GeForce 显卡。RTX 50 系列和RTX 40 系列。

[3] 一文彻底读懂：英伟达GPU分类、架构演进和参数解析
Quadro系列是英伟达专业级GPU产品线，针对商业和专业应用领域进行了优化。常见的产品型号如NVIDIA RTX A6000、A5000等。 Quadro GPU具备强大的计算能力、大 ...


In [45]:
REACT_PROMPT_TEMPLATE = '''
请注意，你是一个有能力调用外部工具的智能助手。

可用工具如下：
{tools}

请严格按照以下的格式进行回应：

Thought: <一句话说明你下一步要做什么>
Action: <只能是以下两种之一>
<tool_name>[<tool_input>]
Finish[<final_answer>]

硬性规则：
1) 你只能输出两行：一行 Thought，一行 Action。不要输出第三行，不要输出列表符号“-”，不要加引号。
2) 当你已经有足够信息回答时, Action 必须是 Finish[...]
4) Action 行必须出现且只能出现一次。Action 行后面不能再写任何文字。

现在，请开始解决以下问题：
Question: {question}
History: {history}
'''


In [46]:
class ReActAgent:
    def __init__(self,llm_client: HelloAgenticsLLM, tool_executor: ToolExecutor, max_steps: int = 5):
        self.llm_client = llm_client
        self.tool_executor = tool_executor
        self.max_steps = max_steps
        self.history = []
    
    def _parse_output(self, text: str):
        thought_match = re.search(r"^Thought[:：]\s*(.*)$", text, re.MULTILINE)
        action_match = re.search(r"^Action[:：]\s*(.*)$", text, re.MULTILINE)

        thought = thought_match.group(1).strip() if thought_match else None
        action_raw = action_match.group(1).strip() if action_match else None
        
        if not action_raw:
            return thought, None
        
        # 在 action_raw 中提取第一个合法动作：Finish[...] 或 ToolName[...]
        m = re.search(r"(Finish\[[^\]]+\]|\w+\[[^\]]+\])", action_raw)
        action = m.group(1) if m else None

        return thought, action

    
    def _parse_action(self, action_text: str):
        match = re.match(r"(\w+)\[(.*)\]", action_text.strip())
        if match:
            return match.group(1), match.group(2)
        return None, None

    def run(self, question: str):
        self.history = []
        current_step = 0

        while current_step < self.max_steps:
            current_step += 1
            print(f"--- 第 {current_step} 步 ---")

            tools_desc = self.tool_executor.getAvaliableTools()
            history_str = '\n'.join(self.history)
            prompt = REACT_PROMPT_TEMPLATE.format(
                tools = tools_desc,
                question=question,
                history=history_str
            )

            messages = [{"role": "user", "content": prompt}]
            response_text = self.llm_client.think(messages=messages)

            if not response_text:
                print("错误：LLM未能返回有效响应。")
                break
                
            though, action = self._parse_output(response_text)

            if though:
                print(f"思考：{though}")
            
            if not action:
                print("警告：未能解析出有效的Action，流程终止。")
                break

            # if action.startswith("Finish"):
            #     final_answer = re.match(r"Finish\[(.*)\]", action).group(1)
            #     print(f"🎉最终答案：{final_answer}")
            #     return final_answer
            if "Finish[" in action:
                final_answer = re.search(r"Finish\[(.*)\]", action).group(1)
                print(f"🎉 最终答案：{final_answer}")
                return final_answer
            
            tool_name, tool_input = self._parse_action(action)
            if not tool_name or not tool_input:
                print(f"❌ 非法 Action，终止：{action}")
                break

            print(f"🎬行动：{tool_name}[{tool_input}]")

            tool_function = self.tool_executor.getTool(tool_name)
            if not tool_function:
                observation = f"错误：未找到名为‘{tool_name}' 的工具。"
            else:
                observation = tool_function(tool_input)

            print(f"👀观察： {observation}")

            self.history.append(f"Action: {action}")
            self.history.append(f"Observation: {observation}")

            import time
            time.sleep(0.8)

        print("已经到最大部署，流程终止")
        return None

In [111]:
llm_client = HelloAgenticsLLM()
tool_executor = ToolExecutor()
search_description = "一个网页搜索引擎。当你需要回答关于时事、事实以及在你的知识库中找不到的信息时，应使用此工具。"
tool_executor.registerTool(name="Search", description=search_description, func=search)
question = "杭州下个周末适合什么活动"
react = ReActAgent(llm_client, tool_executor, max_steps=10)


react.run(question=question)

工具 Search 已注册。
--- 第 1 步 ---
🧠正字调用 GLM-4.1V-Thinking-Flash 模型...
✅ 大模型响应成功：
Thought: 使用Search工具查询杭州下个周末适合的活动推荐
Action: Search[杭州 下个周末 适合的活动 推荐景点 文化活动 休闲]
思考：使用Search工具查询杭州下个周末适合的活动推荐
🎬行动：Search[杭州 下个周末 适合的活动 推荐景点 文化活动 休闲]
🔍 正在执行 [SerpApi] 网页搜索: 杭州 下个周末 适合的活动 推荐景点 文化活动 休闲
👀观察： [1] 杭州市10 大户外活动
5. 杭州私人定制一日游从上海乘坐子弹头列车​ 参加这个全天的私人游览，参观梦幻般的城市杭州，游览杭州的顶级景点，包括西湖和著名的龙津茶场，林隐寺令人惊叹的佛教文化，飞来 ...

[2] 杭州活动汇总_杭州周末去哪玩_杭州近期活动
杭州适合降温天游玩的7个室内场馆推荐！ 阅读2247. 2025-12-14 · “莓”你不行！杭州冬日草莓采摘好去处推荐！ 阅读31214. 2025-12-14 16:00 · 免费居多！杭州周末可以去做的6件 ...

[3] 杭州周末旅游景点推荐_杭州网旅游频道 - 杭州旅游
这几天气温回暖，省内各式新茶陆续上市。选个春光明媚的周末，带上家人出游茶园景区吧——采茶品茗、摄影垂钓，徜徉青山绿水间， ...
--- 第 2 步 ---
🧠正字调用 GLM-4.1V-Thinking-Flash 模型...
✅ 大模型响应成功：
Thought: 整合所获取的杭州下个周末户外、室内及特色活动信息，形成推荐结论
Action: Finish[杭州下个周末适合参与西湖游览、茶园采茶品茗等户外活动，也可选择室内场馆休闲，前往草莓采摘地体验特色采摘，满足不同休闲需求～]
思考：整合所获取的杭州下个周末户外、室内及特色活动信息，形成推荐结论
🎉 最终答案：杭州下个周末适合参与西湖游览、茶园采茶品茗等户外活动，也可选择室内场馆休闲，前往草莓采摘地体验特色采摘，满足不同休闲需求～


'杭州下个周末适合参与西湖游览、茶园采茶品茗等户外活动，也可选择室内场馆休闲，前往草莓采摘地体验特色采摘，满足不同休闲需求～'

## Plan-and-Solve

- 规划阶段 (Planning Phase)： 首先，智能体会接收用户的完整问题。它的第一个任务不是直接去解决问题或调用工具，而是将问题分解，并制定出一个清晰、分步骤的行动计划。这个计划本身就是一次大语言模型的调用产物。
- 执行阶段 (Solving Phase)： 在获得完整的计划后，智能体进入执行阶段。它会严格按照计划中的步骤，逐一执行。每一步的执行都可能是一次独立的 LLM 调用，或者是对上一步结果的加工处理，直到计划中的所有步骤都完成，最终得出答案

规划阶段的目标是让大语言模型接收原始问题，并输出一个清晰、分步骤的行动计划。这个计划必须是结构化的，以便我们的代码可以轻松解析并逐一执行。因此，我们设计的提示词需要明确地告诉模型它的角色和任务，并给出一个输出格式的范例。

In [88]:
PLANNER_PROMPT_TEMPLATE = """
你是一个顶级的AI规划专家。你的任务是将用户提出的复杂问题分解成一个由多个简单步骤组成的行动计划。
请确保计划中的每个步骤都是一个独立的、可执行的子任务，并且严格按照逻辑顺序排列。
你的输出必须是一个Python列表，其中每个元素都是一个描述子任务的字符串。

问题: {question}

请严格按照以下格式输出你的计划:

["步骤1", "步骤2", "步骤3", ...]

"""


In [103]:
import ast

class Planner:
    def __init__(self, llm_client):
        self.llm_client = llm_client
        
    def plan(self, question: str) -> list[str]:
        prompt = PLANNER_PROMPT_TEMPLATE.format(question=question)
        
        message = [{"role": "user", "content": prompt}]
        
        print("--正在生成计划--")
        
        response_text = self.llm_client.think(messages=message)

        print(f"✅ 计划已生成:\n{response_text}")
        
        try:
            # plan_str = response_text.split("'''python")[1].split("'''")[0].strip()
            plan = ast.literal_eval(response_text)
            return plan if isinstance(plan, list) else []
        except (ValueError, SyntaxError, IndexError) as e:
            print(f"❌ 解析计划时出错: {e}")
            print(f"原始响应: {response_text}")
            return []
        except Exception as e:
            print(f"❌ 解析计划时发生未知错误: {e}")
            return []

In [104]:
planner = Planner(llm_client=llm_client)
question = "一个水果店周一卖出了15个苹果。周二卖出的苹果数量是周一的两倍。周三卖出的数量比周二少了5个。请问这三天总共卖出了多少个苹果？"

plan = planner.plan(question)

--正在生成计划--
🧠正字调用 GLM-4.1V-Thinking-Flash 模型...
✅ 大模型响应成功：
["确定周一卖出的苹果数量为15个", "计算周二卖出的苹果数量，即周一数量的两倍（15 × 2）", "计算周三卖出的苹果数量，即周二数量减少5个（周二数量 - 5）", "计算这三天的总销量，将周一、周二和周三的苹果数量相加"]
✅ 计划已生成:
["确定周一卖出的苹果数量为15个", "计算周二卖出的苹果数量，即周一数量的两倍（15 × 2）", "计算周三卖出的苹果数量，即周二数量减少5个（周二数量 - 5）", "计算这三天的总销量，将周一、周二和周三的苹果数量相加"]


In [107]:
EXECUTOR_PROMPT_TEMPLATE = """
你是一位顶级的AI执行专家。你的任务是严格按照给定的计划，一步步地解决问题。
你将收到原始问题、完整的计划、以及到目前为止已经完成的步骤和结果。

你必须以“合法 JSON”的形式输出结果，且只能输出一个 JSON 对象，不要输出任何额外文字。

输出结构必须为：

- 包含字段 "result"，其值为字符串，表示“当前步骤”的最终答案。

规则：
1) 只输出 JSON，不要输出解释性文字。
2) 不要使用 Markdown，不要使用 ```。
3) JSON 结构必须是：{{"result": "<answer>"}}

# 原始问题:
{question}

# 完整计划:
{plan}

# 历史步骤与结果:
{history}

# 当前步骤:
{current_step}

请输出当前步骤的 JSON 结果：
"""


In [94]:
class Executor:
    def __init__(self, llm_client):
        self.llm_client = llm_client
    def execute(self, question: str, plan: list[str]) -> str:
        history = []
        
        print("\n--- 正在执行计划 ---")
        
        for i, step in enumerate(plan):
            print(f"\n-> 正在执行步骤 {1+i}/{len(plan)}: {step}")
            
            prompt = EXECUTOR_PROMPT_TEMPLATE.format(
                question=question,
                plan=plan,
                history=history if history else "无",
                current_step=step
            )

            messages = [{"role": "user", "content":prompt}]
            response_text = self.llm_client.think(messages=messages)
            history += f"步骤 {i+1}: {step}\n结果: {response_text}\n\n"

In [108]:
executor = Executor(llm_client=llm_client)
executor.execute(question, plan)


--- 正在执行计划 ---

-> 正在执行步骤 1/4: 确定周一卖出的苹果数量为15个
🧠正字调用 GLM-4.1V-Thinking-Flash 模型...
✅ 大模型响应成功：
{"result": "15"}


-> 正在执行步骤 2/4: 计算周二卖出的苹果数量，即周一数量的两倍（15 × 2）
🧠正字调用 GLM-4.1V-Thinking-Flash 模型...
✅ 大模型响应成功：
{"result": "30"}


-> 正在执行步骤 3/4: 计算周三卖出的苹果数量，即周二数量减少5个（周二数量 - 5）
🧠正字调用 GLM-4.1V-Thinking-Flash 模型...
✅ 大模型响应成功：
{"result": "25"}


-> 正在执行步骤 4/4: 计算这三天的总销量，将周一、周二和周三的苹果数量相加
🧠正字调用 GLM-4.1V-Thinking-Flash 模型...
✅ 大模型响应成功：
{"result": "70"}



In [96]:
class PlanAndSolveAgent:
    def __init__(self, llm_client):
        
        self.llm_client = llm_client
        self.planner = Planner(self.llm_client)
        self.executor = Executor(self.llm_client)
        
    def run(self, question: str):
        
        print(f"\n--- 开始处理问题 ---\n 问题：{question}")
        
        plan = self.planner.plan(question)
        
        if not plan:
            print("\n ---任务终止--- \n 无法生成有效的行动计划。")
            return
        
        final_answer = self.executor.execute(question, plan)
        
        if not plan:
            print(f"\n --- 任务完成 ---\n 最终答案： {final_answer}")
        

In [109]:
llm_client = HelloAgenticsLLM()
planSolver = PlanAndSolveAgent(llm_client)
question = "一个水果店周一卖出了15个苹果。周二卖出的苹果数量是周一的两倍。周三卖出的数量比周二少了5个。请问这三天总共卖出了多少个苹果？"
planSolver.run(question)


--- 开始处理问题 ---
 问题：一个水果店周一卖出了15个苹果。周二卖出的苹果数量是周一的两倍。周三卖出的数量比周二少了5个。请问这三天总共卖出了多少个苹果？
--正在生成计划--
🧠正字调用 GLM-4.1V-Thinking-Flash 模型...
✅ 大模型响应成功：
[ "步骤1: 确定周一卖出的苹果数量为15个", "步骤2: 计算周二卖出的苹果数量（周一数量的两倍，即15 × 2）", "步骤3: 计算周三卖出的苹果数量（周二数量减少5个，即周二数量 - 5）", "步骤4: 将周一、周二、周三卖出的苹果数量相加，得到总销量" ]
✅ 计划已生成:
[ "步骤1: 确定周一卖出的苹果数量为15个", "步骤2: 计算周二卖出的苹果数量（周一数量的两倍，即15 × 2）", "步骤3: 计算周三卖出的苹果数量（周二数量减少5个，即周二数量 - 5）", "步骤4: 将周一、周二、周三卖出的苹果数量相加，得到总销量" ]

--- 正在执行计划 ---

-> 正在执行步骤 1/4: 步骤1: 确定周一卖出的苹果数量为15个
🧠正字调用 GLM-4.1V-Thinking-Flash 模型...
✅ 大模型响应成功：
{"result": "周一卖出的苹果数量为15个"}

-> 正在执行步骤 2/4: 步骤2: 计算周二卖出的苹果数量（周一数量的两倍，即15 × 2）
🧠正字调用 GLM-4.1V-Thinking-Flash 模型...
✅ 大模型响应成功：
{"result": "周二卖出的苹果数量为30个"}
（这里需要准确反映步骤2的计算结果，周一15个的两倍是30个，所以结果表述为周二卖出30个）
最终输出的JSON应该是包含该结果的，所以：
{"result": "周二卖出的苹果数量为30个"}
不过可能更简洁一点，比如直接说周二卖出30个，所以最终输出为{"result": "周二卖出的苹果数量为30个"}？或者根据历史步骤的格式，可能需要更符合问题描述的结构。哦，问题是步骤2计算周二卖出的苹果数量（周一数量的两倍），所以计算过程是15×2=30，因此结果就是周二卖出30个。所以最终的JSON是{"result": "周二卖出的苹果数量为30个"}？或者更简洁的"周二卖出30个"？但看历史步骤里的格式，历史步骤的结果是包含引号的字符串